<a href="https://colab.research.google.com/github/adelsuh/cs224_final_project/blob/main/graph_structure_aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS224W Final Project: Tutorial on the Augmentation of Graphs in PyG using Structural Augmentations

### Jerry Chan, Jihee Suh, John So

Data augmentation is a widely used technique that leverages existing data to further train a model, improving its performance and generalization. For structured data formats such as images, augmentation methods can be quite straightforward, including operations like cropping, resizing, rotating, and adding noise. These augmentations are useful for reducing overfitting to the training dataset and adding invariance to certain transformations, such as color shifts, different camera models, and even different camera poses.

PyG provides support for dataset augmentations, which primarily inherit from the `torch_geometric.transforms` class. In this Colab, we will cover how augmentations can modify the graph structure to improve performance in inductive graph learning settings.

### Notebook setup: install PyG + torch

In [ ]:
import torch
torch_version = str(torch.__version__)
if "2.4.0" not in torch_version:
  !pip install torch==2.4.0 -q
print(torch_version)

In [ ]:
scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
!pip install torch-scatter -f $scatter_src -q
!pip install torch-sparse -f $sparse_src -q
!pip install torch-geometric -q
!pip install ogb -q

In [ ]:
import os
import random
import numpy as np
import torch

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"using device: {device}")

### Setting up the dataset and tasks

The below code sets up some hyperparameters which will be used in dataloading and training.


In [ ]:
# Model settings
hidden_dim = 128 #@param {type: "integer"}
num_layers = 4 #@param {type: "integer"}
# Training settings
learning_rate = 0.0001 #@param {type: "number"}
num_epochs = 25 #@param {type: "integer"}

# Dataloader settings
batch_size = 32 #@param {type: "integer"}
fan_out = 30 #@param {type: "integer", hint: "Used in neighborhood sampling to sample a subgraph"}
dataloader_num_workers = 2 #@param {type: "integer"}

print(f"""
Running training with the following configuration:
   hidden_dim: {hidden_dim}
   num_layers: {num_layers}
   learning_rate: {learning_rate}
   num_epochs: {num_epochs}
   batch_size: {batch_size}
""")

### Task: Node property prediction with GraphSAGE

For this tutorial, we will consider the [ogbn-arxiv](https://ogb.stanford.edu/docs/nodeprop/) dataset from the Open Graph Benchmark (OGB). This dataset consists of ~170K nodes and ~1.2M directed edges. Each node represents an arXiv CS paper, and each edge represents the citation network between arXiv papers. The prediction task is to predict one of 40 labels for each node (i.e. CS.AI, CS.OS, etc.), given a 128-dimensional feature vector consisting of averaged language embeddings for the arXiv paper.

Run the below block to create the dataset. If this is your first time loading the dataset, it will additionally prompt you to download files.

**Note**: this block loads the dataset into RAM each time it is called! So calling this block multiple times will likely consume all of the notebook's RAM. Take caution.

In [ ]:
from ogb.nodeproppred import PygNodePropPredDataset
dataset = PygNodePropPredDataset(name='ogbn-arxiv', root='./arxiv/')

We'll approach this as an *inductive graph prediction* problem; we want to train one network across many graphs, such that when given new nodes in the graph, or entirely new graphs, our predictor can generalize. To aid with this, the `ogbn-arxiv` dataset provides dataset splits:

- **train split**: CS papers published up until 2017
- **validation split**: CS papers published during 2018
- **test split**: CS papers published during or after 2019.

We'll train on the subgraphs on the train split, then optimize parameters based on performance on the validation split. Lastly, we'll report the performance of methods on the test split.

In [ ]:
split_idx = dataset.get_idx_split()
print(f"""
Summary of the OBGN Arxiv dataset:
  Number of graphs: {len(dataset)}
  Number of features: {dataset.num_features}
  Number of classes: {dataset.num_classes}
  Length of each split:
    Training: {len(split_idx['train'])}
    Validation: {len(split_idx['valid'])}
    Test: {len(split_idx['test'])}
""")

Now, let's create some dataloaders. Training GNNs on graphs with 100k+ node is computationally prohibitive, making full-batch training infeasible. To address this, we can use neighbor sampling, a technique designed for efficient mini-batch training on large graphs by sampling smaller subgraphs. In particular, PyG provides native functionality for this with the `NeighborLoader` class.

The `num_neighbors` parameter controls how neighbors are sampled. It consists of a length $k$ list of integers, and performs $k$ sampling iterations. Starting from a sampled node, NeighborLoader samples `num_neighbors[i]` neighbors from nodes involved in the previous iteration. In our code, we sample $k$-hop neighborhoods, where $k$ is the depth of our GNN. This approach ensures scalability by iteratively constructing smaller, representative sub-graphs while preserving the graph structure necessary for effective learning.

In [ ]:
from torch_geometric.loader import NeighborLoader

data = dataset[0]
train_loader = NeighborLoader(
    data,
    input_nodes=split_idx['train'],
    num_neighbors=[fan_out] * num_layers,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=dataloader_num_workers
)
val_loader = NeighborLoader(
    data,
    input_nodes=split_idx['valid'],
    num_neighbors=[fan_out] * num_layers,
    batch_size=batch_size,
    shuffle=True,
    num_workers=dataloader_num_workers,
)
test_loader = NeighborLoader(
    data,
    input_nodes=split_idx['test'],
    num_neighbors=[fan_out] * num_layers,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)

In [ ]:
print(f"Example batch:")
train_batch = next(iter(train_loader))
print(train_batch)

### Training and Evaluation Utilities

Given this dataset, let's now choose a GNN model. For the purposes of this tutorial, we will choose a simple yet powerful network. [GraphSage](https://arxiv.org/abs/1706.02216) (Hamilton 2017) leverages node features and neighborhood aggregation to learn deep features for nodes. This lends itself to a variety of prediction problems, including our node classification problem.

PyG includes a native implementation of [GraphSAGE](https://pytorch-geometric.readthedocs.io/en/2.5.3/generated/torch_geometric.nn.models.GraphSAGE.html), which we will use to construct our predictor. The predictor consists of a GraphSAGE model to learn deep node embeddings. Then, we pass in a small MLP to output prediction logits for each node.

In [ ]:
from torch_geometric.nn.models import GraphSAGE

input_dim = dataset.num_features

def get_model():
    class GraphSAGENodeClassification(torch.nn.Module):
        def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
            super(GraphSAGENodeClassification, self).__init__()
            self.graph_sage = GraphSAGE(in_channels = input_dim, hidden_channels = hidden_dim, num_layers=num_layers)
            self.cls_head = torch.nn.Sequential(
                torch.nn.Dropout(0.1),
                torch.nn.ReLU(),
                torch.nn.Linear(hidden_dim, num_classes),
            )
            self.loss_fn = torch.nn.CrossEntropyLoss()

        def forward(self, x, edge_index):
            h = self.graph_sage(x, edge_index)
            return self.cls_head(h)

    model = GraphSAGENodeClassification(input_dim, hidden_dim, num_layers, dataset.num_classes)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model.to(device)
    return model, optimizer

Next, we'll define some helper functions, `train_one_epoch` and `test` below. Notably, our functions will take in two arguments:
- `transform`: whether to apply a unary transformation function to the data, such as removing nodes or adding features.
- `filter_output_fn`: whether to apply a binary transformation function to the output. This is intended to handle any extra information introduced by the transformation.


In [ ]:
from tqdm import tqdm

# training process
def train_one_epoch(model,
                    dataloader,
                    optimizer,
                    transform=None,
                    filter_output_fn=None):
    """
    Run one epoch of training on the model on the given dataset.

    Args:
        model (torch.nn.Module): The model to evaluate.
        dataloader (torch.utils.data.DataLoader): The dataloader for the dataset.
        transform: if specified and apply_transform is True, a transformation to apply to each batch
        filter_output_fn: if specified and apply_transform is True, a transformation to apply to the output of each batch.
    """
    model.train()

    # define stats
    total_loss = 0
    total_correct = 0
    num_examples = 0

    for batch in tqdm(dataloader):

        # transform batch if needed
        batch_size = batch.batch_size
        batch = batch.to(device)
        if transform is not None:
          batch = transform(batch)
        # forward pass
        optimizer.zero_grad()
        logits = model(batch.x, batch.edge_index)
        if filter_output_fn is not None:
          logits = filter_output_fn(logits, batch)

        # backward pass
        labels = batch.y.squeeze(-1)

        # use only the supervision nodes; the first batch_size nodes
        logits = logits[:batch_size]
        labels = labels[:batch_size]

        loss = model.loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        # log stats
        total_loss += loss.item() * batch_size
        total_correct += logits.argmax(dim=-1).eq(labels).sum().item()
        num_examples += batch_size

    loss = total_loss / num_examples
    acc = total_correct / num_examples
    return loss, acc

# test process
@torch.no_grad()
def test(model,
         dataloader,
         transform=None,
         filter_output_fn=None,
         apply_transform=True):
    """
    Calculate metrics for the model on the given dataset.

    Args:
        model (torch.nn.Module): The model to evaluate.
        dataloader (torch.utils.data.DataLoader): The dataloader for the dataset.
        apply_transform: whether to use the arguments transform and filter_output_fn.
        transform: if specified and apply_transform is True, a transformation to apply to each batch
        filter_output_fn: if specified and apply_transform is True, a transformation to apply to the output of each batch.
    """
    model.eval()

    # define states
    total_loss = 0
    total_correct = 0
    num_examples = 0

    for batch in tqdm(dataloader):
        # transform batch if needed
        batch_size = batch.batch_size
        batch = batch.to(device)
        if apply_transform and (transform is not None):
          batch = transform(batch)

        # forward pass
        logits = model(batch.x, batch.edge_index)
        if apply_transform and (filter_output_fn is not None):
          logits = filter_output_fn(logits, batch)

        # use only the supervision nodes: the first batch_size nodes
        labels = batch.y.squeeze(-1)

        logits = logits[:batch_size]
        labels = labels[:batch_size]
        loss = model.loss_fn(logits, labels)
        # log stats
        total_loss += loss.item() * batch_size
        total_correct += logits.argmax(dim=-1).eq(labels).sum().item()
        num_examples += batch_size

    loss = total_loss / num_examples
    acc = total_correct / num_examples
    return loss, acc

To train and evaluate the model, call the below `train` function!

In [ ]:
def train(model,
          optimizer,
          num_epochs,
          transform=None,
          filter_output_fn=None,
          apply_transform_at_test=True):
    all_train_acc, all_val_acc, all_test_acc = [], [], []
    best_val_ind, best_val_acc = 0, 0
    for epoch in range(num_epochs):
        print(f'Epoch: {epoch+1:02d}')

        # training
        train_loss, train_acc = train_one_epoch(model,
                                                train_loader,
                                                optimizer,
                                                transform,
                                                filter_output_fn)
        val_loss, val_acc = test(model,
                                 val_loader,
                                 transform,
                                 filter_output_fn=filter_output_fn,
                                 apply_transform=apply_transform_at_test)
        test_loss, test_acc = test(model,
                                   test_loader,
                                   transform,
                                   filter_output_fn=filter_output_fn,
                                   apply_transform=apply_transform_at_test)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_val_ind = epoch

        print(f'Train {train_loss:.4f} ({100.0 * train_acc:.2f}%) | Val {val_loss:.4f} ({100.0 * val_acc:.2f}%) | Test {test_loss:.4f} ({100.0 * test_acc:.2f}%)')

        all_train_acc.append(train_acc)
        all_val_acc.append(val_acc)
        all_test_acc.append(test_acc)

    return {
        'all_train_acc': np.array(all_train_acc),
        'all_val_acc': np.array(all_val_acc),
        'all_test_acc': np.array(all_test_acc),
        'best_val_ind': best_val_ind,
        'model': model
    }

To illustrate some example usage, let's run a baseline. This trains a GraphSAGE network with no graph structure augmentations.

In [ ]:
model, optimizer = get_model()
results = train(model, optimizer, num_epochs=num_epochs)

best_bl_train_acc = results['all_train_acc'][results['best_val_ind']]
best_bl_val_acc = results['all_val_acc'][results['best_val_ind']]
best_bl_test_acc = results['all_test_acc'][results['best_val_ind']]

In [ ]:
import pandas as pd
if not os.path.exists("out"):
  os.mkdir("out")

rows = [{
    "edge_prob": 0,
    "test_acc": best_bl_test_acc,
    "val_acc": best_bl_val_acc
}]

df = pd.DataFrame(rows)
df.to_csv("out/bl_result.csv")

Now, let's plot the training curve.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, Optional

# Set a clean, modern aesthetic
plt.style.use('seaborn-v0_8-pastel')
sns.set_palette("deep")

def plot(x: Optional[np.ndarray] = None,
         y: Dict[str, np.ndarray] = dict(),
         xlabel: str = "",
         ylabel: str = "accuracy"):

  plt.figure(figsize=(6, 4), dpi=300)

  for key, value in y.items():
    if x is not None:
      plt.plot(x, value, label=key)
    else:
      plt.plot(value, label=key)

  plt.grid(True, linestyle='--', linewidth=0.5, color='grey', alpha=0.7)
  plt.title('accuracy')
  if x is not None:
    plt.xlabel(xlabel)
  plt.ylabel(ylabel)

  plt.legend(frameon=True, fancybox=True, framealpha=0.7)
  plt.tight_layout()
  plt.gca().set_facecolor('none')
  plt.gcf().patch.set_alpha(0.0)

  plt.show()

In [ ]:
to_plot = {
    "train": results['all_train_acc'],
    "val": results['all_val_acc'],
}

plot(y=to_plot, xlabel="epoch")

## Graph Structure Augmentation

Modifying the **struture** of the graph is also a powerful way to improve the performance of GNNs. The performance of GNNs is very much related to the structure of the graph.

To better motivate graph structure augmentations, let’s first revisit the core idea of GNNs: message passing. At each layer, nodes aggregate information from their neighbors, gradually building a representation that reflects their local neighborhood structure. In theory, deeper networks should be able to capture broader relationships in the graph, integrating information from distant nodes.

- **Over-smoothing**:    Recall that a GNN with $k$ layers aggregates information from each node's $k$-hop neighborhood. Thus, as the network deepens, node representations increasingly mix, and after many layers, nodes tend to converge to very similar representations. This “blending” means that the network struggles to distinguish between nodes, especially in large, densely connected graphs. In extreme cases, the output becomes almost uniform across all nodes, rendering the GNN ineffective for tasks like classification or clustering.

- **Global relationships**:    While increasing the receptive field of each node by stacking more GNN layers might seem like a solution, it exacerbates the over-smoothing problem noted above, highlighting the trade-offs between depth and effective information propagation.

Graph structure augmentations tackle these challenges head-on by altering the graph’s connectivity, introducing extra nodes and/or edges. to improve the flow of information across the graph, mitigate over-smoothing, and enable GNNs to better capture both local and global patterns.

### Half Hop

Half-Hop (introduced in [Azabou 2023](https://arxiv.org/abs/2308.09198)) enhances message passing in neural networks by inserting intermediate "slow nodes" between connected nodes in a graph. This approach mitigates over-smoothing and improves performance, especially in scenarios where neighboring nodes have different labels. The PyG documentation can be found [here](https://pytorch-geometric.readthedocs.io/en/stable/generated/torch_geometric.transforms.HalfHop.html).

First, let's visualize what Half-Hop is doing. First, let's define an example graph:

In [ ]:
import networkx as nx
from torch_geometric.utils import to_networkx
from pylab import show
from torch_geometric.data import Data
import matplotlib.pyplot as plt

num_nodes = 4
simple_x = torch.arange(num_nodes)
simple_x1 = simple_x + 1
simple_x1[-1] = 0
simple_edge_index = torch.stack([simple_x, simple_x1], dim=0)

simple_x = (simple_x + 1) / 4
simple_data = Data(x=simple_x, edge_index=simple_edge_index, num_nodes=num_nodes)
simple_data, simple_data.x[0]

G = to_networkx(simple_data)
plt.figure(figsize=(3,3))
pos = nx.spring_layout(G)
node_color = [simple_data.x[node] for node in G.nodes()]
nx.draw(G,
        pos=pos,
        cmap=plt.get_cmap('coolwarm'),
        node_color=node_color,
        vmin=0,
        vmax=1,
        with_labels=True)
show()

Now, let's apply the HalfHop transformation.

In [ ]:
from torch_geometric.transforms import HalfHop
hh = HalfHop(alpha=0.5, p=1.0)
hh_data = hh(simple_data)

G = to_networkx(hh_data)
plt.figure(figsize=(3,3))
pos = nx.spring_layout(G)

node_color = [hh_data.x[node] for node in G.nodes()]
nx.draw(G,
        pos=pos,
        cmap=plt.get_cmap('coolwarm'),
        node_color=node_color,
        vmin=0,
        vmax=1,
        with_labels=True)
show()

Below, we define the appropriate trasformations to train HalfHop. Note that we need to filter out the slow nodes which HalfHop creates, so that we are left only with the labels of the true graph's nodes.

In [ ]:
import pandas as pd

from torch_geometric.transforms import HalfHop
# blend features from src and dest equally (alpha=0.5).
# add a virtual node to all edges (p=1)

def hh_filter(outputs, batch):
  # note that the labels (i.e. batch.y) do not need to be masked.
  # dummy labels are not added
  return outputs[~batch.slow_node_mask]

probs = [0.1, 0.5, 1.0]
best_hh_train_acc, best_hh_val_acc, best_hh_test_acc = [], [], []
for hh_prob in probs:
    hh_transform = HalfHop(alpha=0.5, p=hh_prob)
    model, optimizer = get_model()
    print(f"Training with half-hop probability of {hh_prob}\n")
    result = train(model,
                   optimizer,
                   num_epochs=num_epochs,
                   transform=hh_transform,
                   filter_output_fn=hh_filter,
                   apply_transform_at_test=True)

    best_ind = result['best_val_ind']
    best_hh_train_acc.append(result['all_train_acc'][best_ind])
    best_hh_val_acc.append(result['all_val_acc'][best_ind])
    best_hh_test_acc.append(result['all_test_acc'][best_ind])

Now, let's plot the result. HalfHop outperforms the baseline marginally.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('out/hh_result.csv', index_col=0)
df.loc[5] = [0, 0.5418, 0.615]

plt.figure(figsize=(8, 6))
sns.set_style("whitegrid")

ax = sns.barplot(
    x='hh_prob',
    y='test_acc',
    data=df,
    color='skyblue',
    alpha=0.8
)

# Add value labels on top of bars
for i in ax.containers:
    ax.bar_label(i, fmt='%.4f', padding=3)

ax.set(
    xlabel='Half Hop Probability',
    ylabel='Test Accuracy',
    title='Test Accuracy vs Half Hop Probability'
)

# Format y-axis as percentages
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.1%}'.format(y)))

# Adjust y limits to focus on the variation
ax.set_ylim(0.53, 0.56)

plt.tight_layout()
sns.despine()

### Virtual Node

VirtualNode (introduced in [Gilmer 2017](https://arxiv.org/abs/1704.01212)) appends a virtual node to the given homogeneous graph that is connected to all other nodes. The virtual node serves as a global scratch space that each node both reads from and writes to in every step of message passing. This allows information to travel long distances during the propagation phase.

Again, let's visualize what VirtualNode does. Here's an example graph:

In [ ]:
import networkx as nx
from torch_geometric.utils import to_networkx
from pylab import show
from torch_geometric.data import Data
import matplotlib.pyplot as plt

num_nodes = 4
simple_x = torch.arange(num_nodes)
simple_x1 = simple_x + 1
simple_x1[-1] = 0
simple_edge_index = torch.stack([simple_x, simple_x1], dim=0)

simple_x = (simple_x + 1) / 4
simple_data = Data(x=simple_x, edge_index=simple_edge_index, num_nodes=num_nodes)

G = to_networkx(simple_data)
plt.figure(figsize=(3,3))
pos = nx.spring_layout(G)
node_color = [simple_data.x[node] for node in G.nodes()]
nx.draw(G,
        pos=pos,
        cmap=plt.get_cmap('coolwarm'),
        vmin=0,
        vmax=1,
        nodelist=G.nodes(),
        node_color=node_color,
        with_labels=True)
show()

Now, we create the virtual node. It is connected bi-directionally to all nodes, and has an initial feature of 0.

In [ ]:
from torch_geometric.transforms import VirtualNode
vn = VirtualNode()
vn_data = vn(simple_data)

G = to_networkx(vn_data)
plt.figure(figsize=(3,3))
pos = nx.spring_layout(G)

node_color = [vn_data.x[node] for node in G.nodes()]
nx.draw(G,
        pos=pos,
        cmap=plt.get_cmap('coolwarm'),
        nodelist=G.nodes(),
        vmin=0,
        vmax=1,
        node_color=node_color,
        with_labels=True)
show()

Now, let's run training! Note that the virtual node transformation adds an extra dummy label and dummy node to both batch.x and batch.labels. It is fine to leave these in.

In [ ]:
import pandas as pd

from torch_geometric.transforms import VirtualNode

def vn_filter(outputs, batch):
  # virtual node adds a dummy label (0), and a dummy node (0)
  return outputs

vn_transform = VirtualNode()
model, optimizer = get_model()
print(f"Training with virtual node\n")
result = train(model,
                optimizer,
                num_epochs=num_epochs,
                transform=vn_transform,
                filter_output_fn=vn_filter,
                apply_transform_at_test=True)

best_ind = result['best_val_ind']
best_vn_train_acc = result['all_train_acc'][best_ind]
best_vn_val_acc = result['all_val_acc'][best_ind]
best_vn_test_acc = result['all_test_acc'][best_ind]

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Create DataFrame with results
df = pd.DataFrame({
    'model': ['baseline', 'virtual node'],
    'test_acc': [best_bl_test_acc, best_vn_test_acc]
})

plt.figure(figsize=(8, 6))
sns.set_style("whitegrid")

ax = sns.barplot(
    x='model',
    y='test_acc',
    data=df,
    color='skyblue',
    alpha=0.8
)

# Add value labels on top of bars
for i in ax.containers:
    ax.bar_label(i, fmt='%.4f', padding=3)

ax.set(
    xlabel='Model Type',
    ylabel='Test Accuracy',
    title='Test Accuracy: Baseline vs Virtual Node'
)

# Format y-axis as percentages
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.1%}'.format(y)))

# Adjust y limits to focus on variation
ymin = min(df['test_acc']) * 0.95
ymax = max(df['test_acc']) * 1.05
ax.set_ylim(ymin, ymax)

plt.tight_layout()
sns.despine()
